Following the Medium article to create tokens to then get embeddings for the vector database representation of the AIS data with question context.

In [1]:
#!pip install openai
import boto3
import os
import pandas as pd

In [ ]:
#! aws s3 cp 's3://mhw-2023/ais-data/raw/aoi=black_sea/blacksea_roi_05152022.csv' '/home/ec2-user/SageMaker/'

In [4]:
from transformers import GPT2TokenizerFast

In [3]:
#!pip3 install transformers

  Using cached transformers-4.18.0-py3-none-any.whl (4.0 MB)
  Using cached huggingface_hub-0.4.0-py3-none-any.whl (67 kB)
  Using cached tokenizers-0.12.1-cp36-cp36m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (6.6 MB)
  Using cached sacremoses-0.0.53-py3-none-any.whl


In [5]:
df = pd.read_csv('blacksea_roi_05152022.csv')

In [9]:
df.shape

(5008713, 11)

In [6]:
df['Blacksea'] = 'POLYGON ((37.94937448100006 44.77463101400008, 37.66711467600004 45.61837073000004, 35.08326373500006 45.429614377000064, 34.23379615600004 44.73996726200005, 28.589714650000076 45.20634161900006, 27.56178749600008 42.61748804500007, 28.073775271000045 41.59304610300006, 28.766117433000034 41.29860173600008, 26.64395197400006 40.70215620300007, 26.984909837000032 39.98724455400003, 29.723571383000035 40.65816164000006, 29.800561868000045 41.18609639600004, 31.261605648000057 41.02172795900003, 32.39624108500004 41.68103522900003, 33.58409428600004 42.02199309200006, 34.80494340900003 41.96699988900008, 36.124780298000076 41.23009095900005, 37.752579129000026 40.90013173600005, 40.22884816000004 40.79561406800008, 41.83498433900007 41.783090386000026, 41.513757103000046 42.71108017800003, 37.94937448100006 44.77463101400008))'

In [10]:
df

,lat,long,sog,cog,heading,rot,nav_stt,timestamp,mmsi,shiptype,Blacksea
0,40.757500,29.506120,10.3,267.399994,262.0,0.0,0.0,2022-02-16 13:27:00,244010871,75,"POLYGON ((37.94937448100006 44.77463101400008,..."
1,40.741760,29.377020,7.5,263.500000,264.0,-127.0,1.0,2022-02-16 22:53:00,677017000,75,"POLYGON ((37.94937448100006 44.77463101400008,..."
2,40.741306,29.316190,8.3,265.700012,265.0,0.0,1.0,2022-02-16 23:13:00,677017000,75,"POLYGON ((37.94937448100006 44.77463101400008,..."
3,40.826780,28.885027,0.1,5.800000,213.0,0.0,0.0,2022-02-16 07:17:00,371999000,75,"POLYGON ((37.94937448100006 44.77463101400008,..."
4,40.826767,28.886114,0.1,271.000000,186.0,0.0,0.0,2022-02-16 06:57:00,371999000,75,"POLYGON ((37.94937448100006 44.77463101400008,..."
...,...,...,...,...,...,...,...,...,...,...,...
5008708,44.740303,36.463665,7.2,178.800003,511.0,-128.0,0.0,2022-05-14 21:34:00,273332800,75,"POLYGON ((37.94937448100006 44.77463101400008,..."
5008709,42.126457,34.905710,11.5,86.500000,81.0,0.0,0.0,2022-05-14 20:59:00,271000910,75,"POLYGON ((37.94937448100006 44.77463101400008,..."
5008710,42.129265,34.927593,11.0,55.500000,43.0,-127.0,0.0,2022-05-14 21:04:00,271000910,75,"POLYGON ((37.94937448100006 44.77463101400008,..."
5008711,44.631634,36.426067,8.3,187.600006,181.0,5.0,0.0,2022-05-14 21:06:00,667001512,75,"POLYGON ((37.94937448100006 44.77463101400008,..."


In [7]:
from transformers import GPT2TokenizerFast
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
tokenizer("Hello world")['input_ids']
tokenizer(" Hello world")['input_ids']

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

[18435, 995]

In [20]:
for i in df.iterrows():
    f'''Which vessels, where vessel id is {i.get('mmsi')}\
    are in the area of the Blacksea where the area of the Blacksea is{i.get('Blacksea')}\
    and the vessel location is {i.get('lat')} {i.get('long')} on 5/15/2022, where date is {i.get(timestamp)}'''    

AttributeError: 'tuple' object has no attribute 'get'

In [19]:
df = df.assign(questions = lambda x: 'Which vessels, where vessel id is'+x['mmsi']+
    'are in the area of the Blacksea where the area of the Blacksea is'+x['Blacksea']+
    'and the vessel location is'+x['lat']+x['long']+'on 05/15/2022, where the date is'+x['timestamp'])

ValueError: NumExpr 2 does not support Unicode as a dtype.

Which vessels, where vessel id is df['mmsi'] are in the area of the Blacksea where the area of the Blacksea is df['Blacksea'] and the vessel location is df['lat'], df['long'] on 05/14/2022,where the date is df['timestamp']?

What is the average speed and direction of the oil tankers in the Blacksea where the speed is df['sog'] in knots and the df['cog'] is the direction of travel in degrees and oil tankers are df['shiptype']==75?